In [ ]:
import numpy as np
import pydot
from IPython.display import SVG, display
from matplotlib import gridspec
from matplotlib import pyplot as plt
from pydrake.geometry.optimization import GraphOfConvexSetsOptions, HPolyhedron, Point
from pydrake.planning import GcsTrajectoryOptimization

# GCS Trajectory Optimization with Derivative Constraints

GCS Trajectory Optimization provides a powerful tool for obtaining globally-optimal solution. With the more powerful solver, you might have be a bit more careful what you wish for! In particular, the interplay between shortest paths and derivative constraints can be quite subtle. This simple notebook tries to make that point.

Consider a very simple GCSTrajOpt problem with (effectively) two regions -- one to the left of the origin (in light blue), and one to the right of the origin (light green). We'll put the start at the bottom left, the goal at the bottom right, and use an edge constraint to ensure that the trajectory passes through the origin.

In [ ]:
def PlotEnvironment():
    plt.axis("square")
    plt.fill([-1, 0, 0, -1], [-1, -1, 1, 1], "lightblue", alpha=0.5)
    plt.fill([0, 1, 1, 0], [-1, -1, 1, 1], "lightgreen", alpha=0.5)
    plt.plot([-1, 0, 1], [-1, 0, -1], "r*")
    plt.xlim([-1.25, 1.25])
    plt.ylim([-1.25, 1.25])
    plt.xlabel("x")
    plt.ylabel("y")
    plt.xticks()
    plt.yticks()
    plt.grid(1)


def PlotSolution(traj, result):
    assert result.is_success()

    gs = gridspec.GridSpec(2, 1, height_ratios=[4, 1])  # 4:1 ratio for height
    plt.subplot(gs[0])
    PlotEnvironment()

    plt.plot(*traj.value(traj.start_time()), "kx")
    plt.plot(*traj.value(traj.end_time()), "kx")
    times = np.linspace(traj.start_time(), traj.end_time(), 1000)
    waypoints = traj.vector_values(times)
    plt.plot(*waypoints, "b", zorder=5)
    for seg in [traj.segment(i) for i in range(traj.get_number_of_segments())]:
        plt.plot(seg.control_points()[0], seg.control_points()[1], "ro")

    plt.subplot(gs[1])
    plt.plot(times, waypoints.T)
    plt.xlabel("time (s)")
    plt.legend(["x", "y"])


PlotEnvironment()


def AddRegionsAndEdges(trajopt, order=4):
    left = trajopt.AddRegions(
        [HPolyhedron.MakeBox([-1, -1], [0, 1])], order=order, name="left"
    )
    right = trajopt.AddRegions(
        [HPolyhedron.MakeBox([0, -1], [1, 1])], order=order, name="right"
    )
    source = trajopt.AddRegions([Point([-1, -1])], order=0, name="source")
    target = trajopt.AddRegions([Point([1, -1])], order=0, name="target")
    trajopt.AddEdges(source, left)
    trajopt.AddEdges(right, target)
    e = trajopt.AddEdges(left, right).Edges()[0]
    e.AddConstraint(e.xu()[-2] == 0).evaluator().set_description(
        "left-right edge: y = 0"
    )
    return source, left, right, target


trajopt = GcsTrajectoryOptimization(2)
source, left, right, target = AddRegionsAndEdges(trajopt, order=4)

display(
    SVG(
        pydot.graph_from_dot_data(trajopt.graph_of_convex_sets().GetGraphvizString())[
            0
        ].create_svg()
    )
)

## Minimum distance, no derivative constraints

Naturally, the shortest path given this setup is the straight line from the start to the origin, then the origin to the goal. Solving GcsTrajOpt without any derivative constraints recovers this solution.

In [ ]:
trajopt = GcsTrajectoryOptimization(2)
source, left, right, target = AddRegionsAndEdges(trajopt, order=4)

trajopt.AddPathLengthCost()
[traj, result] = trajopt.SolvePath(source, target)

PlotSolution(traj, result)

Notice the time duration in the solution. Why did it choose that for the time duration? Our problem formulation so far has underspecified the timing. It could be infinitely fast or infinitely slow.  This can lead to strange numerical artifacts. An alternative formulation would be to solve for minimum time, subject to some velocity bounds.

## Minimum distance, with velocity limits

In [ ]:
trajopt = GcsTrajectoryOptimization(2)
source, left, right, target = AddRegionsAndEdges(trajopt, order=4)

trajopt.AddPathLengthCost()
trajopt.AddVelocityBounds([-1, -1], [1, 1])
[traj, result] = trajopt.SolvePath(source, target)

PlotSolution(traj, result)

## Minimum time, with velocity limits

In [ ]:
trajopt = GcsTrajectoryOptimization(2)
source, left, right, target = AddRegionsAndEdges(trajopt, order=4)

trajopt.AddVelocityBounds([-1, -1], [1, 1])
trajopt.AddTimeCost()
[traj, result] = trajopt.SolvePath(source, target)

PlotSolution(traj, result)

## Minimum time, velocity limits and C(1) continuity

In [ ]:
trajopt = GcsTrajectoryOptimization(2)
source, left, right, target = AddRegionsAndEdges(trajopt, order=4)

trajopt.AddVelocityBounds([-1, -1], [1, 1])
trajopt.AddContinuityConstraints(continuity_order=1)
trajopt.AddTimeCost()
[traj, result] = trajopt.SolvePath(source, target)

PlotSolution(traj, result)

Now here's the tricky one.  If you ask for minimum distance + continuity constraints, you might be surprised with what you get.

## Minimum distance with C(1) continuity

In [ ]:
trajopt = GcsTrajectoryOptimization(2)
source, left, right, target = AddRegionsAndEdges(trajopt, order=4)

trajopt.AddVelocityBounds([-1, -1], [1, 1])
trajopt.AddContinuityConstraints(continuity_order=1)
trajopt.AddPathLengthCost()
options = GraphOfConvexSetsOptions()
# NOTE: I have to disable rounding... otherwise SNOPT will fail.
options.max_rounded_paths = 0
[traj, result] = trajopt.SolvePath(source, target, options)

PlotSolution(traj, result)

The solution in time looks reasonable. The solution in x-y looks a little surprising... that doesn't look like a smooth curve?  What's going on here?
The optimal solution puts multiple control points immediately on top of each other. So the velocity is continuous in time, but could change very rapidly in space. Adding higher derivative limits does not help... the trajectory can slow down in time, but still change direction very rapidly in space in order minimize the distance.

This feels like a bad formulation... it drives the solver towards a solution that is numerically bad. In fact, SNOPT fails to solve it... so I've actually had to disable the rounding step in the code cell above.

## Takeaways

Be careful with your formulation. If your optimal solution is arbitrarily bad numerically, then you might need to rethink your formulation.